<a href="https://colab.research.google.com/github/cnn22/SingerSongwriter/blob/main/TaylorSwift_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# !pip install spotipy

# 1. Gather Data

## 1.1 Get Lyrics

In [ ]:
#get all of taylor swift's albums
url = "https://taylor-swift-api.sarbo.workers.dev/albums"
response = requests.get(url)

if response.status_code == 200:
  data = response.json()
  albums = pd.DataFrame(data)
  # print(albums)
else:
  print(f'Failed to retrieve data. Status code: {response.status_code}')

In [ ]:
# gets all of taylor's songs for all albums
url = 'https://taylor-swift-api.sarbo.workers.dev/songs'
response = requests.get(url)

if response.status_code == 200:
  data = response.json()
  songs = pd.DataFrame(data)
  # print(songs)
else:
  print(f'Failed to retrieve data. Status code: {response.status_code}')

In [ ]:
#get lyrics
lyrics_url = 'https://taylor-swift-api.sarbo.workers.dev/lyrics/'
lyrics_df = pd.DataFrame(columns=['song_title', 'lyrics'])

for index, row in songs.iterrows():
  song = row['song_id']
  url = f'{lyrics_url}{song}'
  response = requests.get(url)

  if response.status_code == 200:
    data = response.json()
    lyrics = data['lyrics']
    song_title = data['song_title']
    new_row = pd.DataFrame({'song_title': [song_title], 'lyrics': [lyrics]})
    lyrics_df = pd.concat([lyrics_df, new_row], ignore_index = True)
  else:
    print(f'Failed to retrieve data. Status code: {response.status_code}')

In [ ]:
lyrics = pd.merge(songs, lyrics_df, left_on ='title', right_on = 'song_title', how = 'inner').drop(columns=['song_title'])
albums = albums.rename(columns={'title':'album_title'})
lyrics = pd.merge(lyrics, albums, on = 'album_id', how = 'inner')
# lyrics.to_csv('all_taylor_swift_lyrics.csv', index = True)


## 1.2 Get song attributes

In [120]:
songs = pd.read_csv('all_taylor_swift_lyrics.csv', index_col = False)

In [123]:
songs['Artist'] = 'Taylor Swift'
songs['SpotifyID'] = None
songs["Tempo"] = None
songs["Loudness"] = None
songs["Key"] = None
songs["isExplicit"] = None
songs["Danceability"] = None
songs["Energy"] = None
songs["Liveness"] = None
songs["Duration"] = None
songs["Popularity"] = None

In [148]:
cid = ""
secret = ""

#initiating Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [108]:
#getSongID takes track's name, track's Artist, and track results
#iterate through the track results to get the name in what spotify returned and
# test those results against what song and artist we have in the data frame
def getSongID(trackName, trackArtist, trackResults):
    for track in trackResults['tracks']['items']:
        if (track['name'].lower() == trackName.lower()) & (track['artists'][0]['name'].lower() == trackArtist.lower()):
            return track['id']
    return None

In [109]:
#getExplicit function that takes in a songID and returns if the song is explicit or not
def getExplicit(songID):
    return sp.track(songID)['explicit']

In [110]:
#getTrackAttributes function that takes in a songID and returns song attributes such as, Tempo, Loudness, Key and Duration
def getTrackAttributes(songID):
    return sp.audio_features(tracks=songID)[0]

In [112]:
def getPopularity(songID):
  return sp.track(songID)['popularity']

In [113]:
#getSongKey function that takes in a pitch class and returns the key. Reference: https://en.wikipedia.org/wiki/Pitch_class
def getSongKey(pitchClass):
    if pitchClass == 0:
        return 'C'
    elif pitchClass == 1:
        return 'C#, Db'
    elif pitchClass == 2:
        return 'D'
    elif pitchClass == 3:
        return 'D#, Eb'
    elif pitchClass == 4:
        return 'E'
    elif pitchClass == 5:
        return 'F'
    elif pitchClass == 6:
        return 'F#, Gb'
    elif pitchClass == 7:
        return 'G'
    elif pitchClass == 8:
        return 'G#, Ab'
    elif pitchClass == 9:
        return 'A'
    elif pitchClass == 10:
        return 'A# Bb'
    elif pitchClass == 11:
        return 'B'
    else: return None

In [129]:
for index, row in songs.head().iterrows():
  trackResults = sp.search(q=row['title'], type='track', market= 'US', limit=10,offset=0)
  songID = getSongID(row['title'], row['Artist'], trackResults)
  # print(songID)
  songs.SpotifyID.iloc[index] = songID

<ipython-input-129-ebd82787d332>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songs.SpotifyID.iloc[index] = songID


In [143]:
songs.loc[songs['title'] == 'tis the damn season', 'SpotifyID'] = '4GBkffrtA51p17JH35irGA'
songs.loc[songs['title'] == "Soon You'll Get Better", 'SpotifyID'] = '4AYtqFyFbX0Xkc2wtcygTr'
songs.loc[songs['title'] == "It's Nice To Have A Friend", 'SpotifyID'] = '1SmiQ65iSAbPto6gPFlBYm'
songs.loc[songs['title'] == "Don't Blame Me", 'SpotifyID'] = '1R0a2iXumgCiFb7HEZ7gUE'
songs.loc[songs['title'] == "New Year's Day", 'SpotifyID'] = '7F5oktn5YOsR9eR5YsFtqb'
songs.loc[songs['title'] == "Question…?", 'SpotifyID'] = '0heeNYlwOGuUSe7TgUD27B'
songs.loc[songs['title'] == "Snow on the Beach", 'SpotifyID'] = '4zmKGsrXjLmljb5fTaBTot'
songs.loc[songs['title'] == "So It Goes…", 'SpotifyID'] = '5PxFv9yJEg9dxvbZggykro'
songs.loc[songs['title'] == "it's time to go", 'SpotifyID'] = '6RvmQShxVQwJ0AWMtP6JoU'

In [150]:
songs = songs[~songs.SpotifyID.isnull()]

In [153]:
#iteratte through songs in songs data frame (which we pulled from billboard.com) to grab song attributes
#using Spotify's API for things like, isExplicit, tempo, loudness, duration, release date.

for index, row in songs.iterrows():
    #get explicit attribute from Get Track API
    explicitValue = getExplicit(row['SpotifyID'])
    attributes = getTrackAttributes(row['SpotifyID'])

    #filling in values
    songs.at[index, 'Tempo'] = attributes['tempo']
    songs.at[index, 'Loudness'] = attributes['loudness']
    songs.at[index, 'Key'] = getSongKey(attributes['key'])
    songs.at[index, 'isExplicit'] = explicitValue
    songs.at[index, 'Danceability'] = attributes['danceability']
    songs.at[index, 'Energy'] = attributes['energy']
    songs.at[index, 'Liveness']= attributes['liveness']
    songs.at[index, 'Duration'] = attributes['duration_ms']/1000 #converting from milliseconds to seconds
    songs.at[index, 'Popularity'] = getPopularity(row['SpotifyID'])

In [156]:
songs.to_csv('TAYLOR_SWIFT_FINAL_DATASET.csv', index = False)

# 2. Sentiment Analysis

In [159]:
import pandas as pd
import pickle as pk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [158]:
# !pip install vaderSentiment

In [ ]:
# songs = pd.read_csv('TAYLOR_SWIFT_FINAL_DATASET.csv')

In [162]:
analyzer = SentimentIntensityAnalyzer()

sentiment_scores = []

for index, row in songs.iterrows():
  song = row['lyrics']
  sentiment = analyzer.polarity_scores(song)
  sentiment_scores.append({
      'Title': row['title'],
      'Album': row['album_title'],
      'Positive': sentiment['pos'],
      'Neutral': sentiment['neu'],
      'Negative': sentiment['neg'],
      'Compound': sentiment['compound']
  })

In [ ]:
for sentiment_data in sentiment_scores:
    print(f"Album: {sentiment_data['Album']}")
    print(f"Track: {sentiment_data['Title']}")
    print(f"Positive: {sentiment_data['Positive']}")
    print(f"Neutral: {sentiment_data['Neutral']}")
    print(f"Negative: {sentiment_data['Negative']}")
    print(f"Compound: {sentiment_data['Compound']}")
    print("\n")  # Add a newline for better readability between song data

In [168]:
sentiment_df = pd.DataFrame(sentiment_scores)

In [169]:
sentiment_df

,Title,Album,Positive,Neutral,Negative,Compound
0,Blank Space,1989,0.164,0.688,0.149,0.8537
1,Style,1989,0.096,0.898,0.007,0.9894
2,Out Of The Woods,1989,0.227,0.766,0.007,0.9995
3,All You Had to Do Was Stay,1989,0.128,0.850,0.022,0.9932
4,Shake It Off,1989,0.089,0.500,0.411,-0.9998
...,...,...,...,...,...,...
129,Bigger Than the Whole Sky,Midnights,0.000,0.957,0.043,-0.8665
130,Paris,Midnights,0.220,0.734,0.046,0.9971
131,Glitch,Midnights,0.116,0.862,0.022,0.9750
132,Dear Reader,Midnights,0.205,0.729,0.066,0.9921


In [170]:
songs.columns

Index(['song_id', 'title', 'album_id', 'lyrics', 'album_title', 'release_date',
       'Artist', 'SpotifyID', 'Tempo', 'Loudness', 'Key', 'isExplicit',
       'Danceability', 'Energy', 'Liveness', 'Duration', 'Popularity'],
      dtype='object')